# Setup
1. Mount the colab to our drive.
2. Clone repo, install dependencies and check PyTorch and GPU.

In [ ]:
# Mount this script to my drive
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/2022_趨勢_機器學習比賽/')

In [2]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.1-176-gaa7a0e9 torch 1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)


Setup complete ✅ (4 CPUs, 25.5 GB RAM, 38.8/166.8 GB disk)


# Weights & Biases Logging

Weights & Biases (W&B) is now integrated with YOLOv5 for real-time visualization and cloud logging of training runs. This allows for better run comparison and introspection, as well improved visibility and collaboration among team members. To enable W&B logging install wandb, and then train normally (you will be guided setup on first use).


$ pip install wandb
During training you will see live updates at https://wandb.ai, and you can create Detailed Reports of your results using the W&B Reports tool.

In [ ]:
!pip install wandb;
! wandb login

# It is time to train our data!

In [6]:
!python train.py --img 960 --batch 16 --epochs 300 --freeze 10 --data STAS.yaml --weights yolov5x.pt --hyp YC_hyp.yaml

wandb: Currently logged in as: ss13974268. Use `wandb login --relogin` to force relogin
train: weights=yolov5x.pt, cfg=, data=STAS.yaml, hyp=YC_hyp.yaml, epochs=300, batch_size=16, imgsz=960, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[10], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 66 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-176-gaa7a0e9 torch 1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.05, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=

# Check the training result
Training results are automatically logged to Tensorboard and CSV as results.csv, which is plotted as results.png (below) after training completes. You can also plot any results.csv file manually:

In [ ]:
from utils.plots import plot_results
plot_results('./runs/train/v5x_6/results.csv')  # plot 'results.csv' as 'results.png'

# Testing
Using the public dataset to test the best result.

In [11]:
!python val.py --weights ./runs/train/v5x_8/weights/best.pt --data STAS.yaml --img 960 --half --task test --save-json --conf-thres 0.05 --iou-thres 0.6

val: data=/content/drive/.shortcut-targets-by-id/1Uuw4VSQfRJRYLOPrTRRUTwCyws9rcF3i/2022_趨勢_機器學習比賽/yolov5/data/STAS.yaml, weights=['./runs/train/v5x_8/weights/best.pt'], batch_size=32, imgsz=960, conf_thres=0.05, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-176-gaa7a0e9 torch 1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model summary: 444 layers, 86173414 parameters, 0 gradients, 204.0 GFLOPs
test: Scanning '/content/drive/.shortcut-targets-by-id/1Uuw4VSQfRJRYLOPrTRRUTwCyws9rcF3i/2022_趨勢_機器學習比賽/OBJ_Train_Datasets/STAS_YOLOv5/labels/test.cache' images and labels... 0 found, 315 missing, 0 empty, 0 corrupt: 100% 315/315 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 10/10 [00:15<00:00,  1.54s/it]
           

# Convert the output format
Covert the testing result to the required format for submitting.


In [13]:
import json

file_location = 'runs/val/v5x_8_on_test/best_predictions.json'
output_location = 'runs/val/v5x_8_on_test/test_submit.json'
f = open(file_location,'r')
data = json.load(f)

if type(data[0]['image_id'])==int: 
  for i in data:
    num = i['image_id']
    i['image_id']=f'{num:08}'

d={}

for labels in data:
  if labels['score']<0.05:
    continue
  f_name = labels['image_id']+'.jpg'
  bbox = labels['bbox']
  xmin=int(bbox[0])
  ymin=int(bbox[1])
  xmax=int(bbox[0]+bbox[2]-1)
  ymax=int(bbox[1]+bbox[3]-1)
  bbox = [xmin,ymin,xmax,ymax,labels['score']]
  if f_name not in d:
    d[f_name] = []
  d[f_name].append(bbox)

  with open(output_location, 'w') as convert_file:
     convert_file.write(json.dumps(d))

# Label the validation set
We label the validation result and the ground true label to check the fit result.

In [ ]:
# creates a ScandirIterator aliased as files
def read_val_set(img_file_path):
  STAS=[]
  with os.scandir(img_file_path) as files:
    # loops through each file in the directory
      for file in files:
          if file.name.endswith('.jpg'):
            # adds only the image files to the list
              STAS.append(file.name)
  return STAS

In [ ]:
import numpy as np
import json
import cv2
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

# The location of each files
YOLO_output_location = 'runs/val/v5x_6_val_IoU_0.006/test_submit.json' # YOLO output
val_img_location='../OBJ_Train_Datasets/STAS_YOLOv5/images/val/' # val images

# Read the number list of the val set
nums_list = read_val_set(val_img_location)
for i in range(len(a)):
  nums_list[i] = nums_list[i].split('.')[0]

# Read the list of the YOLO output for the val set
f = open(YOLO_output_location,'r')
YOLO_output_data = json.load(f)  # The order of the bbox: [xmin, ymin, xmax, ymax]

for t_num in nums_list:
  print(f'dealing with {t_num}')
  # Read the ground true labels for each image
  ann_file = f'../OBJ_Train_Datasets/Train_Annotations/{t_num}.xml'
  tree = ET.parse(ann_file)
  root = tree.getroot()
  gound_true_dict={} # Iinitialize the dictionary
  count=0               # Initialize the counting number, 
                        # which will represent the number of STAS in each image.
  for obj in root.findall('object'):
    xmin=int(obj.find('bndbox').find('xmin').text)
    ymin=int(obj.find('bndbox').find('ymin').text)
    xmax=int(obj.find('bndbox').find('xmax').text)
    ymax=int(obj.find('bndbox').find('ymax').text)
    gound_true_dict[count]=[(xmin,ymin),(xmax,ymax)]
    count+=1

  # Read the YOLO output labels
  YOLO_output_t_num = YOLO_output_data[f'{t_num}.jpg']
  YOLO_output_dict={} # Iinitialize the dictionary
  count=0               # Initialize the counting number
  for obj in YOLO_output_t_num:
    xmin=int(obj[0])
    ymin=int(obj[1])
    xmax=int(obj[2])
    ymax=int(obj[3])
    YOLO_output_dict[count]=[xmin,ymin,xmax,ymax,obj[4]]
    count+=1
    
  # Read the image
  img_file=f'../OBJ_Train_Datasets/Train_Images/{t_num}.jpg'
  img = cv2.imread(img_file)

  # Plot the ground true labels
  for obj_num in gound_true_dict.keys():
    vertex1=gound_true_dict[obj_num][0]
    vertex2=gound_true_dict[obj_num][1]
    cv2.rectangle(img,vertex1,vertex2,(0,0,0),3) 

  # Plot the YOLO output labels
  for obj_num in YOLO_output_dict.keys():
    xmin=YOLO_output_dict[obj_num][0]
    ymin=YOLO_output_dict[obj_num][1]
    xmax=YOLO_output_dict[obj_num][2]
    ymax=YOLO_output_dict[obj_num][3]
    cv2.rectangle(img,(xmin, ymin),(xmax, ymax),(0,255,0),3) 
    cv2.putText(img, str(YOLO_output_dict[obj_num][4]), (xmin, ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

  # Save the result
  output_file = f'runs/YC_test/val_v5x_6_IoU0.006/{t_num}.jpg'
  cv2.imwrite(output_file,img)
